In [ ]:
from callstats import CallRecorder

In [ ]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """

   styles = open(css_file_path, "r").read()
   s = '<style>%s</style>' % styles     
   return HTML(s)

_set_css_style('custom/custom.css')

<div style='width:20em'><img src='img/logo-igm.png'></div>
<div style='font-size:larger'><strong>Algorithmique et programmation 2</strong><br>
L1 Mathématiques - L1 Informatique<br>
Semestre 2
</div>

# Récursivité

## Appels de fonctions et pile d'appels

### Une petite histoire

- Vous allez chez G. en lisant un bouquin  
    - Arrivée à la station, vous commencez à consulter le plan  
        - R. appelle pour vous proposer un cinéma demain  
            - Vous consultez le programme  
            - Vous trouvez un film qui vous plaît  
        - Vous vous mettez d'accord avec R.  
    - Vous finissez de chercher votre ligne de métro  
- Vous reprenez votre lecture à la bonne page

Un être humain sait faire ça (un peu), **mais une machine** ?

### Les appels de fonctions
  
-   Un programme utilise des fonctions (pré-définies ou non)
-   Certaines fonctions font appel à d'autres fonctions
-   Au moment de l'appel, l'exécution de la fonction appelante s'interrompt
-   Après un appel, l'exécution doit reprendre au bon endroit, et avec les
    bonnes valeurs des variables  

<img src='img/appelrec.png' width=50%></img>

### Exemple : quelques tâches simples

**Exercice :**
Décrivez l'exécution du programme suivant, l'évolution de la pile
d'appels et les affichages

In [ ]:
def pgcd(a, b):
    while b:
        a, b = b, a % b
    return a

def simplifier(fraction):
    d = pgcd(fraction[0], fraction[1])
    res = (fraction[0] // d, fraction[1] // d)
    return res

def ajouter(frac1, frac2):
    denom = frac1[1] * frac2[1]
    num = frac1[0] * frac2[1] + frac2[0] * frac1[1]
    res = simplifier((num, denom))
    return res

frac = ajouter((1, 3), (1, 6))

In [ ]:
frac

<iframe width="800" height="500" frameborder="1" src="https://pythontutor.com/iframe-embed.html#code=def%20pgcd%28a,%20b%29%3A%0A%20%20%20%20while%20b%3A%0A%20%20%20%20%20%20%20%20a,%20b%20%3D%20b,%20a%20%25%20b%0A%20%20%20%20return%20a%0A%0Adef%20simplifier%28fraction%29%3A%0A%20%20%20%20d%20%3D%20pgcd%28fraction%5B0%5D,%20fraction%5B1%5D%29%0A%20%20%20%20res%20%3D%20%28fraction%5B0%5D%20//%20d,%20fraction%5B1%5D%20//%20d%29%0A%20%20%20%20return%20res%0A%0Adef%20ajouter%28frac1,%20frac2%29%3A%0A%20%20%20%20denom%20%3D%20frac1%5B1%5D%20*%20frac2%5B1%5D%0A%20%20%20%20num%20%3D%20frac1%5B0%5D%20*%20frac2%5B1%5D%20%2B%20frac2%5B0%5D%20*%20frac1%5B1%5D%0A%20%20%20%20res%20%3D%20simplifier%28%28num,%20denom%29%29%0A%20%20%20%20return%20res%0A%0Afrac%20%3D%20ajouter%28%281,%203%29,%20%281,%206%29%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=true&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

In [ ]:
@CallRecorder
def pgcd(a, b):
    while b:
        a, b = b, a % b
    return a

@CallRecorder
def simplifier(fraction):
    d = pgcd(fraction[0], fraction[1])
    res = (fraction[0] // d, fraction[1] // d)
    return res

@CallRecorder
def ajouter(frac1, frac2):
    denom = frac1[1] * frac2[1]
    num = frac1[0] * frac2[1] + frac2[0] * frac1[1]
    res = simplifier((num, denom))
    return res

In [ ]:
my_call = ajouter((1, 3), (1, 6))
print(my_call.trace())

In [ ]:
my_call.graph()

### Pile d'appels

**Pile d'appels :** liste ordonnée de tâches en cours
  
-   « Fond » de la pile : première tâche commencée (la plus ancienne)
-   « Sommet » de la pile : dernière tâche commencée (la plus récente)  

  

  Chaque élément de pile <!-- $1 -->
  contient :
  
  -   Valeur des paramètres de la fonction en cours
  -   Position du site d'appel (instruction à exécuter au retour)
  -   Variables locales à la fonction, état de l'interpréteur
  

### Fonctionnement de la pile

  À chaque nouvel **appel** :
  -   La fonction appelante évalue chaque paramètre et met sur la pile (des références à) ces valeurs
  -   L'appelante sauvegarde son état (ou *contexte*) pour
    pouvoir reprendre son exécution une fois l'appel terminé
  -   L'interpréteur donne la main à la fonction appelée
  -   L'appelée démarre son exécution, en plaçant ses éventuelles
    variables locales au sommet de la pile  

À chaque nouveau **retour** de fonction :
  
-   L'appelée dépose à un endroit convenu sa valeur de retour
-   L'appelée détruit (« dépile ») toutes ses données locales
-   L'interpréteur rend la main à la fonction appelante
-   L'appelante récupère la valeur de retour et reprend son exécution

Les détails (qui fait quoi, où, comment) dépendent de l'implémentation

### Exemple : périmètre d'un polygone

In [ ]:
def distance(p, q):
    '''Renvoie la longueur du côté
    defini par p et q.'''
    dx = (p[0] - q[0])
    dy = (p[1] - q[1])
    res = (dx ** 2 + dy ** 2) ** 0.5
    return res

def perimetre(P):
    '''Renvoie le périmètre du polygone 
    défini par la liste P de points.'''
    retval = 0
    for i in range(-1, len(P) - 1):
        d = distance(P[i], P[i+1])
        retval += d
    return retval

p = perimetre([(0, 0), (0, 1), (1, 1), (1, 0)])

In [ ]:
p

<iframe width="800" height="500" frameborder="1" src="https://pythontutor.com/iframe-embed.html#code=def%20distance%28p,%20q%29%3A%0A%20%20%20%20'''Renvoie%20la%20longueur%20du%20c%C3%B4t%C3%A9%0A%20%20%20%20defini%20par%20p%20et%20q.'''%0A%20%20%20%20dx%20%3D%20%28p%5B0%5D%20-%20q%5B0%5D%29%0A%20%20%20%20dy%20%3D%20%28p%5B1%5D%20-%20q%5B1%5D%29%0A%20%20%20%20res%20%3D%20%28dx%20**%202%20%2B%20dy%20**%202%29%20**%200.5%0A%20%20%20%20return%20res%0A%0Adef%20perimetre%28P%29%3A%0A%20%20%20%20'''Renvoie%20le%20p%C3%A9rim%C3%A8tre%20du%20polygone%20%0A%20%20%20%20d%C3%A9fini%20par%20la%20liste%20P%20de%20points.'''%0A%20%20%20%20retval%20%3D%200%0A%20%20%20%20for%20i%20in%20range%28-1,%20len%28P%29%20-%201%29%3A%0A%20%20%20%20%20%20%20%20d%20%3D%20distance%28P%5Bi%5D,%20P%5Bi%2B1%5D%29%0A%20%20%20%20%20%20%20%20retval%20%2B%3D%20d%0A%20%20%20%20return%20retval%0A%0Ap%20%3D%20perimetre%28%5B%280,%200%29,%20%280,%201%29,%20%281,%201%29,%20%281,%200%29%5D%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=true&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

In [ ]:
@CallRecorder
def distance(p, q):
    '''Renvoie la longueur du côté
    defini par p et q.'''
    dx = (p[0] - q[0])
    dy = (p[1] - q[1])
    res = (dx ** 2 + dy ** 2) ** 0.5
    return res

@CallRecorder
def perimetre(P):
    '''Renvoie le périmètre du polygone 
    défini par la liste P de points.'''
    retval = 0
    for i in range(-1, len(P) - 1):
        d = distance(P[i], P[i+1])
        retval += d
    return retval

my_call = perimetre([(0, 0), (0, 1), (1, 1), (1, 0)])
print(my_call.trace())

## Programmation récursive

### Principe

C'est le même principe que le raisonnement ou la définition par récurrence en math !

**Idée fondamentale :**
Puisque une fonction peut en appeler une autre, pourquoi une fonction ne pourrait-elle pas s'appeler elle-même ?

### Exemple : somme et produit 

Voici une proposition de définition « récurrente » de la somme et du produit de deux entiers positifs. 

Pour tous entiers positifs $a$ et $b$, on a:

$$
a + b = 
\begin{cases}
  a & \text{si } b = 0,
  \\
  \big(a + (b - 1)\big) + 1 & \text{sinon.}
\end{cases}
$$
$$
a \times b = 
\begin{cases}
  0 & \text{si } b = 0,
  \\
  \big(a \times (b - 1)\big) + a & \text{sinon.}
\end{cases}
$$

Voici une fonction Python qui imite la définition du produit :

In [ ]:
def produit(a, b):
    """Calcule le produit de a par b (entier positif), 
    sans utiliser l'opérateur « * » !"""
    if b == 0:
        res = 0
        return res
    else:
        res = a + produit(a, b-1)
        return res
    
p = produit(4, 3)

In [ ]:
p

<iframe width="800" height="500" frameborder="1" src="https://pythontutor.com/iframe-embed.html#code=def%20produit%28a,%20b%29%3A%0A%20%20%20%20%22%22%22Calcule%20le%20produit%20de%20a%20par%20b%20%28entier%20positif%29,%20%0A%20%20%20%20sans%20utiliser%20l'op%C3%A9rateur%20%C2%AB%20*%20%C2%BB%20!%22%22%22%0A%20%20%20%20if%20b%20%3D%3D%200%3A%0A%20%20%20%20%20%20%20%20res%20%3D%200%0A%20%20%20%20%20%20%20%20return%20res%0A%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20res%20%3D%20a%20%2B%20produit%28a,%20b-1%29%0A%20%20%20%20%20%20%20%20return%20res%0A%20%20%20%20%0Ap%20%3D%20produit%284,%203%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=true&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

Autre visualisation possible :

In [ ]:
@CallRecorder
def produit(a, b):
    """Calcule le produit de a par b (entier positif), 
    sans utiliser l'opérateur « * » !"""
    if b == 0:
        return 0
    else:
        return a + produit(a, b-1)
    
my_call = produit(4, 3)
print(my_call.trace())

**Exercice :** programmer de manière semblable une fonction récursive calculant $a + b$.

In [ ]:
def somme(a, b):
    """Calcule la somme de a et b (entier positif), 
    avec seulement (+ 1) !"""
    if b == 0:
        return a
    else:
        return 1 + somme(a, b-1)
    
somme(4, 3)

In [ ]:
@CallRecorder
def somme(a, b):
    """Calcule la somme de a et b (entier positif), 
    avec seulement (+ 1) !"""
    if b == 0:
        return a
    else:
        return 1 + somme(a, b-1)
    
my_call = somme(4, 3)
print(my_call.trace())

### Exemple: factorielle

Rappel: la **factorielle de $n$** est définie par :

$$ 
n! = \begin{cases}
  1 \text{ si $n = 0$}\\
  n\times(n-1)\cdot(n-2)\cdots 2\cdot1 \text{ sinon}
\end{cases}
$$

Calcul itératif de $n!$ :

In [ ]:
def facto(n):
    '''
    >>> facto(5) == 5*4*3*2*1
    True
    '''
    f = 1
    for i in range(2, n + 1):
        f *= i
    return f

facto(5)

Ou sous forme récursive :

$$
n! = \begin{cases}
      1 \text{ si $n = 0$}\\
      n \times (n-1)! \text{ sinon}
    \end{cases}
$$

Calcul récursif de $n!$ :

In [ ]:
def facto(n):
    """
    Attention, n entier >=0 sinon je plante.
    """
    if n == 0:
        return 1
    else:
        return n * facto(n - 1)
    
facto(5)

In [ ]:
@CallRecorder
def facto(n):
    if n == 0:
        return 1
    else:
        return n * facto(n - 1)
    
print(facto(5).trace())

### Exemple : puissance

On a déjà vu comment calculer $a^n=\overbrace{a\times a\times \cdots\times a}^{\text{n termes}}$, avec le cas particulier $a^0=1$

Calcul itératif de $a^n$ :

In [ ]:
def puiss(a, n):
    p = 1
    for i in range(1, n + 1):
        p *= a
    return p

puiss(2, 3)

On peut aussi définir $a^n$, pour $n\in \mathbb{N}$, de manière récursive:

$$
a^n = \begin{cases}
1 & \text{ si } n=0\\
a \times a^{n-1} & \text{ sinon}
\end{cases}
$$

Calcul récursif de $a^n$ :

In [ ]:
@CallRecorder
def puissR(a, n):
    if n == 0:
        return 1
    else:
        return a * puissR(a, n - 1)
    
print(puissR(2, 3).trace())

### Exemple : pgcd

Le plus grand diviseur commun à deux entiers positifs ou nuls $a$ et $b$ peut être défini comme :

$$
\mathrm{pgcd}(a, b) = \begin{cases}a & \text{ si $b = 0$,}\\\mathrm{pgcd}(b, a \bmod b) & \text{ sinon}\end{cases}
$$

In [ ]:
def pgcd(a, b):
    if b == 0:
        return a
    else:
        return pgcd(b, a % b)

pgcd(2*5*7, 2*3*5*11)

In [ ]:
@CallRecorder
def pgcd(a, b):
    if b == 0:
        return a
    else:
        return pgcd(b, a % b)

call = pgcd(2*5*7, 2*3*5*11)
print(call.trace())

### Méthode générale

1. Pour les instances les plus simples ("cas de base"), donner la solution directe
1. Pour les cas complexes, se ramener à une ou plusieurs instances "plus simples" du même problème
1. Combiner les solutions des instances plus simples pour résoudre le problème initial

"Plus simple" : varie selon le contexte. Exemples :
-   $P$ et $P'$ sont des naturels et $P'<P$
-   $P$ est une liste et $P'$ est une liste plus courte
-   $P = (a, b)$ et $P' = (a', b')$ sont des couples de nombres et $a' < a$ ou $(a' = a$ et $b' < b)$
-   $P = (a, b)$ et $P' = (a', b')$ sont des couples de nombres et $\mathrm{abs}(b'-a') < \mathrm{abs}(b-a)$
-   $\ldots$

De manière générale "plus simple" veut dire "plus proche du cas de base"

#### Fonction récursive typique :

```python
def f(P):  # P = parametre(s)
    if test(P):
        # bloc sans appel recursif : cas de base
        return ...
    else:
        # bloc avec appel(s) recursif(s) utilisant
        # des parametres "plus simples"
        return ...
```

## Erreurs fréquentes

### Conditions d'arrêt

In [ ]:
def fact(n):
    return n * fact(n-1)

fact(4)

-   Erreur observée : RecursionError (profondeur max de récursion dépassée)
-   Explication : oubli du cas d'arrêt (on va passer dans les valeurs négatives de n)
-   Solution : ajouter un cas d'arrêt

In [ ]:
def fact(n):
    if n == 0 :
        return 1
    else:
        return n * fact(n-1)

fact(4)

### Conditions d'arrêt et appels récursifs

In [ ]:
def f(n): # n naturel
    if n == 0:
        print("!")
    else:
        print("*", end="")
        f(n)
        
f(4)

-   Erreur observée : idem
-   Explication : l'appel récursif ne se rapproche pas du cas de base
-   Solution : on rajoute -1 dans l'appel récursif

In [ ]:
def f(n): # n naturel
    if n == 0:
        print("!")
    else:
        print("*", end="")
        f(n-1)
        
f(10)

### Limite de capacité de la pile

Chaque appel récursif stocke des informations sur la pile d'appel ("bloc" d'exécution de la fonction, en anglais *frame*), donc il est nécessaire de limiter la taille totale de la pile.

In [ ]:
res = fact(2958)

In [ ]:
res = fact(3000)

-   La fonction est correcte, mais erreur si dépassement de la hauteur maximale de la pile
-   *Réglable, mais dans une certaine limite...*

In [ ]:
from sys import setrecursionlimit
setrecursionlimit(10000)
f(9500)

### Exercice

Qu'affichent les appels de fonctions suivants ?

In [ ]:
def fonction1(n):
    if n == 0:
        print(n)
    else:
        print(n, end=' ')
        fonction1(n-1)
        
fonction1(10)

In [ ]:
def fonction2(n):
    if n == 0:
        print(n, end=' ')
    else:
        fonction2(n-1)
        print(n, end=' ')
        
fonction2(10)

In [ ]:
def fonction3(n):
    if n == 0:
        print(n, end=' ')
    else:
        print(n, end=' ')
        fonction3(n-1)
        print(n, end=' ')
        
fonction3(10)

### Exemple : termine-t-elle ?

Pour un certain entier $a$ donné, on définit la suite de Syracuse $(S^a_n)_{n \geq 0}$ de premier terme $a$ par $S^a_0 = a$ et pour tout $n \geq 1$, 

$$
\begin{cases}
S^a_n = S^a_{n-1} / 2 & \text{ si $S^a_{n-1}$ est pair} \\
S^a_n = 3 \cdot S^a_{n-1} +1 & \text{ sinon }
\end{cases}
$$

Exemples :
- $S^5 : (5, 16, 8, 4, 2, 1, 4, 2, 1, ...)$
- $S^7 : (7, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, ...)$

On cherche à savoir quel est le plus petit rang $n$ tel que $S^a_n = 1$.

In [ ]:
def suivant(terme):
    if terme % 2 == 0:
        return terme // 2
    else:
        return 3*terme + 1    

def temps(a):
    if a == 1:
        print(1)
        return 0
    else:
        print(a, end=' ')
        s = suivant(a)
        return 1 + temps(s)

In [ ]:
temps(5)

In [ ]:
temps(7)

In [ ]:
for i in range(1,10):
    print(f"temps de vol à partir de {i} : {temps(i)}")
    print()

-   Fonction `suivant` : terme suivant de la suite de Syracuse
-   Temps de vol : résultat de `temps(n)`
-   *Personne* n'a encore réussi à prouver qu'elle termine toujours !
-   L'évolution du temps de vol de la suite fait des choses inattentues... (*source : Wikipedia*)
-   Plus de détails : [Collatz conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture)

<img src='https://upload.wikimedia.org/wikipedia/commons/b/b9/Collatz-stopping-time.svg' width='50%'></img>

### Exercice

Pour convertir un nombre $n$ en base 2, on peut :

1. Calculer le quotient $q$ et le reste $r$ de $n$ par 2
2. Convertir $q$ en base 2
3. Ajouter $r$ comme dernier chiffre au résultat obtenu

Écrire une fonction récursive convertissant $n$ en base 2 (renvoie une liste de *bits*), puis dans une base $b$ quelconque.

In [ ]:
def decimal_vers_binaire(n):    
    if n < 2:
        return [n]
    else:
        q, r = divmod(n, 2)  # étape 1
        res = decimal_vers_binaire(q)  # étape 2
        res.append(r)
        return res

In [ ]:
decimal_vers_binaire(3422)

In [ ]:
def decimal_vers_base(n, b):    
    if n < b:
        return [n]
    else:
        q, r = divmod(n, b)  # étape 1
        res = decimal_vers_base(q, b)  # étape 2
        res.append(r)
        return res

In [ ]:
decimal_vers_base(3422, 2)

In [ ]:
decimal_vers_base(3422, 8)

### Exercice

La suite de Fibonacci $(F_n)_{n \geq 0}$ est définie par :

$$
\begin{cases}
F_0 = F_1 = 1\\
F_n = F_{n-1} + F_{n-2} & \text{ pour tout } n \geq 2
\end{cases}
$$

Écrire une fonction récursive calculant le $n$-ème terme $F_n$ de la suite de Fibonacci

In [ ]:
def fibonacci(n):
    if n <= 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [ ]:
for i in range(40):
    print(fibonacci(i))

Combien d'appels récursifs cette fonction réalise-t-elle au total pour $n = 5$ ? Pour $n = 10$ ? On peut compter à la main, ou bien demander à la fonction de compter...

In [ ]:
def fibonacci(n):
    if n <= 1:
        return 1, 1
    else:
        a, cpt1 = fibonacci(n-1)
        b, cpt2 = fibonacci(n-2)
        return a + b, cpt1 + cpt2 + 1

In [ ]:
fibonacci(9)

In [ ]:
fibonacci(35)

In [ ]:
fibonacci(40)  # long !!

Visualisons l'ensemble des appels, organisé sous la forme d'un arbre :

In [ ]:
@CallRecorder
def f(n):
    if n <= 1:
        return 1
    else:
        return f(n-1) + f(n-2)

In [ ]:
f(5).graph(verbose_labels=True)

Le problème est que certains appels sont répétés un très grand nombre de fois (on calcule plusieurs fois la même chose...). On dira dans la suite du cours que cette fonction a une *complexité exponentielle* (ce qui est énorme).

In [ ]:
f(15).render(verbose_labels=False, prog='neato')